In [3]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()
driver.get('https://www.wissports.net/game/show/33030961?subseason=763824')
driver.find_element_by_id('tab_gs3_game_stats').click()

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=100.0.4896.127)


In [3]:

team_stats = driver.find_element_by_class_name('StatWidgetHolder')
team_names = driver.find_element_by_class_name('team_box')
my_data = team_stats.text.strip()
my_teams = team_names.text.strip()


In [51]:
my_teams_data = my_teams.split('\n')
game_dict = {}
points_score = []
teams = []
for data in my_teams_data:
    points_score.append(data.rsplit(' ', 1)[1])
    teams.append(data.rsplit(' ', 1)[0])
game_overview = {}
game_overview['Team_Name'] = teams
game_overview['Points_Scored'] = points_score
game_overview['Game_ID'] = [33030961, 33030961]
game = pd.DataFrame(game_overview)
game

,Team_Name,Points_Scored,Game_ID
0,Cedar Grove-Belgium,53,33030961
1,Oostburg,64,33030961


In [5]:
import re
game_stats = my_data.replace('\n', ' ').replace('SCORING STATS', '').replace('REBOUNDS STATS', '').replace('MISC STATS', '').strip()
game_stats_only = re.findall('[0-9]+', game_stats)
game_stats_headers = re.findall(r'\D+', game_stats)
stat_headers = []
for stat in game_stats_headers:
    this_stat = stat.strip()
    if this_stat != '':
        stat_headers.append(this_stat)

game_stats_grouped = [game_stats_only[stat : stat + 2] for stat in range(0, len(game_stats_only), 2)]

In [26]:
game_stats_grouped

[['19', '19'],
 ['42', '45'],
 ['20', '10'],
 ['24', '16'],
 ['10', '5'],
 ['20', '20'],
 ['64', '53'],
 ['8', '0'],
 ['25', '0'],
 ['33', '0'],
 ['16', '0'],
 ['18', '0'],
 ['3', '0'],
 ['1', '0'],
 ['15', '0']]

In [39]:
import pandas as pd
game_stats = {}
for stat in stat_headers:
    team_dict = {}
    this_index = stat_headers.index(stat)
    team_dict[stat] = game_stats_grouped[this_index]
    game_stats.update(team_dict)

my_frame = pd.DataFrame(game_stats)
game_stats


{'Field Goals Made': ['19', '19'],
 'Field Goals Attempted': ['42', '45'],
 'Free Throws Made': ['20', '10'],
 'Free Throws Attempted': ['24', '16'],
 'Three Point Made': ['10', '5'],
 'Three Point Attempted': ['20', '20'],
 'Total Points': ['64', '53'],
 'Offensive Rebounds': ['8', '0'],
 'Defensive Rebounds': ['25', '0'],
 'Total Rebounds': ['33', '0'],
 'Assists': ['16', '0'],
 'Turnovers': ['18', '0'],
 'Steals': ['3', '0'],
 'Blocks': ['1', '0'],
 'Fouls': ['15', '0']}

In [52]:
full_game = pd.concat([game, my_frame], axis = 1)
full_game

,Team_Name,Points_Scored,Game_ID,Field Goals Made,Field Goals Attempted,Free Throws Made,Free Throws Attempted,Three Point Made,Three Point Attempted,Total Points,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Assists,Turnovers,Steals,Blocks,Fouls
0,Cedar Grove-Belgium,53,33030961,19,42,20,24,10,20,64,8,25,33,16,18,3,1,15
1,Oostburg,64,33030961,19,45,10,16,5,20,53,0,0,0,0,0,0,0,0


In [1]:
import time
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import pandas as pd 
import re
import requests 
from bs4 import BeautifulSoup as bs 


In [55]:
site = r'https://www.wissports.net/schedule/day/league_instance/173330/2022/11/18?subseason=827150&referrer=7291983'
page = requests.get(site)
soup = bs(page.content, 'html.parser')
x = soup.find_all('tr', id = lambda x: x and x.startswith('game_list_row_'))

In [56]:
x[0]

<tr class="odd scheduled compactGameList" id="game_list_row_37401446">
<td>
<div class="scheduleListTeam">
<a class="teamName" href="https://www.wissports.net/page/show/7293038-westosha-central?subseason=827150">Westosha Central</a>
<span class="grayed"> (0-0)</span>
</div>
</td>
<td><div> - </div></td>
<td>
<div class="scheduleListTeam">
<a class="teamName" href="https://www.wissports.net/page/show/7293052-platteville?subseason=827150">Platteville</a>
<span class="grayed"> (0-0)</span>
</div>
</td>
<td><div> - </div></td>
<td>
<div class="scheduleListTeam">
      Beaver Dam Classic
    </div>
</td>
<td class="nowrap">
<a class="game_link_referrer" href="https://www.wissports.net/game/show/37401446?subseason=827150">
<span>10:00 AM CST</span>
</a> </td>
<td>-</td>
</tr>

In [57]:
games = []
for result in x:
    game = str(result).split('>')[0].replace('"', '')[-8:]
    games.append(game)

In [58]:
games

['37401446',
 '37401427',
 '37401432',
 '36750725',
 '37401437',
 '37334800',
 '36747325',
 '36747595',
 '36748006',
 '36748082',
 '36748128',
 '36748724',
 '36748982',
 '36749206',
 '36749399',
 '36750132',
 '36750144',
 '36750383',
 '36750523',
 '36750551',
 '36750954',
 '36750999',
 '36751494',
 '36751815',
 '36751891',
 '36752067',
 '36752101',
 '36752254',
 '36752407',
 '37350525',
 '37402587',
 '37424666',
 '36748585',
 '36748818',
 '36748924',
 '36749074',
 '36749346',
 '36749714',
 '36749813',
 '36750101',
 '36750120',
 '36750198',
 '36750236',
 '36750290',
 '36750654',
 '36750946',
 '36751519',
 '36751580',
 '36751696',
 '36751733',
 '36751789',
 '36752110',
 '36748305',
 '36748314',
 '36748346',
 '36748440',
 '36748861',
 '36749134',
 '36749755',
 '36749887',
 '36751094',
 '36752501',
 '37084693',
 '37090698']

In [59]:
import time
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import pandas as pd 
import re
from selenium.webdriver.common.by import By

total_start = time.perf_counter()
full_frames = []
succes_count = 0
error_count = 0
for this in games:
    try:
        start = time.perf_counter()
        driver = webdriver.Chrome()
        search = 'https://www.wissports.net/game/show/' + str(this) + '?subseason=827150'
        driver.get(search)
        driver.maximize_window()
        time.sleep(5)
        driver.find_element(By.ID, 'tab_gs3_game_stats').click()
        time.sleep(10)
        team_stats = driver.find_element(By.CLASS_NAME, 'StatWidgetHolder')
        team_names = driver.find_element(By.CLASS_NAME, 'team_box')
        my_data = team_stats.text.strip()
        my_teams = team_names.text.strip()
        driver.close()
        my_teams_data = my_teams.split('\n')
        game_dict = {}
        points_score = []
        teams = []
        for data in my_teams_data:
            points_score.append(data.rsplit(' ', 1)[1])
            teams.append(data.rsplit(' ', 1)[0])
        game_overview = {}
        game_overview['Team_Name'] = teams
        game_overview['Points_Scored'] = points_score
        game_overview['Game_ID'] = [str(this), str(this)]
        game = pd.DataFrame(game_overview)
        game_stats = my_data.replace('\n', ' ').replace('SCORING STATS', '').replace('REBOUNDS STATS', '').replace('MISC STATS', '').strip()
        game_stats_only = re.findall('[0-9]+', game_stats)
        game_stats_headers = re.findall(r'\D+', game_stats)
        stat_headers = []
        for stat in game_stats_headers:
            this_stat = stat.strip()
            if this_stat != '':
                stat_headers.append(this_stat)

        game_stats_grouped = [game_stats_only[stat : stat + 2] for stat in range(0, len(game_stats_only), 2)]
        game_stats = {}
        for stat in stat_headers:
            team_dict = {}
            this_index = stat_headers.index(stat)
            team_dict[stat] = game_stats_grouped[this_index]
            game_stats.update(team_dict)

        my_frame = pd.DataFrame(game_stats)
        full_game = pd.concat([game, my_frame], axis = 1)
        full_frames.append(full_game)
        end = time.perf_counter()
        succes_count += 1
        print(str(this) + ': Time taken, ' + str(end - start) + ' success')
    except Exception as e:
        print(e)
        try:
            driver.close()
        except Exception:
            continue
        error_count +=1
total_end = time.perf_counter()
print(str(total_end - total_start))
print('Errors: ' + str(error_count))
print('Success: ' + str(succes_count))

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="tab_gs3_game_stats"]"}
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B3ACD3+2075859]
	Ordinal0 [0x00ACEE61+1633889]
	Ordinal0 [0x009CB7BD+571325]
	Ordinal0 [0x009FAC2F+764975]
	Ordinal0 [0x009FAE1B+765467]
	Ordinal0 [0x00A2D0F2+970994]
	Ordinal0 [0x00A17364+881508]
	Ordinal0 [0x00A2B56A+963946]
	Ordinal0 [0x00A17136+880950]
	Ordinal0 [0x009EFEFD+720637]
	Ordinal0 [0x009F0F3F+724799]
	GetHandleVerifier [0x00DEEED2+2769538]
	GetHandleVerifier [0x00DE0D95+2711877]
	GetHandleVerifier [0x00BCA03A+521194]
	GetHandleVerifier [0x00BC8DA0+516432]
	Ordinal0 [0x00AD682C+1665068]
	Ordinal0 [0x00ADB128+1683752]
	Ordinal0 [0x00ADB215+1683989]
	Ordinal0 [0x00AE6484+1729668]
	BaseThreadInitThunk [0x7701FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77727BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77727B8E+238]

Message: no such element: Unable to locate element: {"method":"c

In [54]:
all_games = pd.concat(full_frames)
all_games.to_csv(r'C:\Users\Jake\Documents\GitHub\DataPlayGround2\csv\WIS_GBB_GAME_DATA\11_17_2022.csv', index = False)


In [17]:
driver = webdriver.Chrome()
search = 'https://www.wissports.net/game/show/' + str(33023420) + '?subseason=763824'
driver.get(search)
driver.find_element_by_id('tab_gs3_game_stats').click()


In [21]:
team_stats = driver.find_element_by_class_name('StatWidgetHolder')
team_names = driver.find_element_by_class_name('team_box')
my_data = team_stats.text.strip()
my_data

'SCORING STATS\nField Goals Made\n8\n0\nField Goals Attempted\n32\n0\nFree Throws Made\n4\n0\nFree Throws Attempted\n20\n0\nThree Point Made\n1\n0\nThree Point Attempted\n3\n0\nTotal Points\n21\n0\nREBOUNDS STATS\nOffensive Rebounds\n11\n0\nDefensive Rebounds\n24\n0\nTotal Rebounds\n36\n0\nMISC STATS\nAssists\n3\n0\nTurnovers\n24\n0\nSteals\n4\n0\nBlocks\n2\n0\nFouls\n0\n0\nTechnical Fouls\n0\n0'

In [156]:
import requests
from bs4 import BeautifulSoup as bs
last = 34129053
url = 'https://www.wissports.net/game/show/{}?subseason=763824'
data = requests.get(url)
first = 33030961
#first = 34129052
#params = {
    #"id" : "#tab_gs3_game_stats_content",
 #   "class": "tab+menu-tab z6 loaded"
#}
my_range = range(first, last + 1)
header = {'class' : 'basketball minimal user_mode gameShow mobile'}
for game in my_range:
        this_url = url.format(game)
        data = requests.get(this_url)
        if data.status_code == 200:
                with open(r'C:\Users\Jake\Documents\Data\{}.html'.format(game), 'w+') as f:
                        f.write(data.text)

#root = html.fromstring(page.content)
# <div id="GameShowContent" class=" game_show_nextgen">    

KeyboardInterrupt: 

In [3]:
import re
from bs4 import BeautifulSoup as bs
data = r'C:\Users\Jake\Documents\Data\33030957.html'
soup = bs(data.text, 'html5lib')
games = soup.find_all('table')
scores = soup.find_all('tbody')
scores = str(scores).split('\n')
my_teams = []
first_half = []
second_half = []
field_goals = []
ft = []
threes = []
assists =[]
turnovers = []
steals = []
blocks = []
fouls = []
pattern = '>(.*?)<'
for line in scores:
        if (('"team">')) in line:
                my_teams.append(re.search(pattern, line).group(1))
        elif (("pts_1")) in line:
                first_half.append(re.search(pattern, line).group(1))
        elif (("pts_2")) in line:
                second_half.append(re.search(pattern, line).group(1))

final = list(zip(my_teams, first_half, second_half))
final = soup.find_all('thead')
final = str(final).split('</th>')
for line in final:
        if (('<th title="Field Goals Made">\n')) in line:
                field_goals.append(line.split('\n')[2])
        elif (('<th title="Free Throws Made">\n')) in line:
                ft.append(line)
        elif (('<th title="Three Point Made">\n')) in line:
                threes.append(line)
        elif (('<th title="Assists">\n')) in line:
               assists.append(line)
        elif (('<th title="Turnovers">\n')) in line:
                turnovers.append(line)
        elif (('<th title="Steals">\n')) in line:
                steals.append(line)
        elif (('<th title="Blocks">\n')) in line:
                blocks.append(line)
        elif (('<th title="Fouls">\n')) in line:
                fouls.append(line)

field_goals


AttributeError: 'str' object has no attribute 'text'